In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210220.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice ],Rank your movie choices. You don't have to do a full ranking. [Eighth ],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice],What theme should we explore next week? [Ninth Choice]
0,2/16/2021 18:50:14,Raise the Red Lantern (1991),Quills (2000),The Enigma of Kaspar Hauser (1974),Black Knight (2001),Waterloo (1970),The Tin Drum (1979),Murder on the Orient Express (1974),The Favourite (2018),ROM COMS,OOPS ALL STEVE BUSCEMI,[Default to Nicolas Cage],ANIMALS/CREATURES THAT TALK,EURO-CRIMES,MIAMI,I'M WITH THE BAND,SEX PERVERT MOVIES,1980S ACTION
1,2/16/2021 18:50:34,Murder on the Orient Express (1974),The Favourite (2018),Raise the Red Lantern (1991),Quills (2000),Black Knight (2001),The Enigma of Kaspar Hauser (1974),The Tin Drum (1979),Waterloo (1970),ROM COMS,EURO-CRIMES,MIAMI,SEX PERVERT MOVIES,I'M WITH THE BAND,[Default to Nicolas Cage],1980S ACTION,OOPS ALL STEVE BUSCEMI,ANIMALS/CREATURES THAT TALK
2,2/16/2021 18:54:13,The Favourite (2018),Murder on the Orient Express (1974),Raise the Red Lantern (1991),Waterloo (1970),Black Knight (2001),NaN,NaN,NaN,1980S ACTION,EURO-CRIMES,MIAMI,[Default to Nicolas Cage],OOPS ALL STEVE BUSCEMI,NaN,NaN,NaN,NaN
3,2/16/2021 19:02:49,Waterloo (1970),Quills (2000),The Enigma of Kaspar Hauser (1974),Raise the Red Lantern (1991),The Tin Drum (1979),Murder on the Orient Express (1974),The Favourite (2018),Black Knight (2001),[Default to Nicolas Cage],1980S ACTION,MIAMI,OOPS ALL STEVE BUSCEMI,SEX PERVERT MOVIES,I'M WITH THE BAND,ROM COMS,EURO-CRIMES,ANIMALS/CREATURES THAT TALK
4,2/16/2021 19:09:07,The Enigma of Kaspar Hauser (1974),Raise the Red Lantern (1991),The Tin Drum (1979),Waterloo (1970),Murder on the Orient Express (1974),NaN,NaN,NaN,EURO-CRIMES,SEX PERVERT MOVIES,I'M WITH THE BAND,MIAMI,1980S ACTION,ROM COMS,ANIMALS/CREATURES THAT TALK,[Default to Nicolas Cage],OOPS ALL STEVE BUSCEMI
5,2/16/2021 20:22:14,Black Knight (2001),The Favourite (2018),Waterloo (1970),NaN,NaN,NaN,NaN,NaN,ANIMALS/CREATURES THAT TALK,MIAMI,OOPS ALL STEVE BUSCEMI,ROM COMS,1980S ACTION,NaN,NaN,NaN,NaN
6,2/17/2021 5:52:14,The Favourite (2018),Waterloo (1970),Murder on the Orient Express (1974),Black Knight (2001),Raise the Red Lantern (1991),Quills (2000),The Tin Drum (1979),The Enigma of Kaspar Hauser (1974),ANIMALS/CREATURES THAT TALK,I'M WITH THE BAND,ROM COMS,SEX PERVERT MOVIES,NaN,NaN,NaN,NaN,NaN
7,2/17/2021 11:10:41,The Tin Drum (1979),Waterloo (1970),Raise the Red Lantern (1991),The Enigma of Kaspar Hauser (1974),Black Knight (2001),The Favourite (2018),Murder on the Orient Express (1974),Quills (2000),SEX PERVERT MOVIES,MIAMI,EURO-CRIMES,1980S ACTION,OOPS ALL STEVE BUSCEMI,I'M WITH THE BAND,ROM COMS,ANIMALS/CREATURES THAT TALK,[Default to Nicolas Cage]
8,2/17/2021 14:51:16,Raise the Red Lantern (1991),Quills (2000),The Tin Drum (1979),Waterloo (1970),The Enigma of Kaspar Hauser (1974),Black Knight (2001),Murder on the Orient Express (1974),The Favourite (2018),MIAMI,SEX PERVERT MOVIES,OOPS ALL STEVE BUSCEMI,ANIMALS/CREATURES THAT TALK,EURO-CRIMES,I'M WITH THE BAND,ROM COMS,1980S ACTI

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Raise the Red Lantern (1991)', 'Quills (2000)',
       'The Enigma of Kaspar Hauser (1974)', 'Black Knight (2001)',
       'Waterloo (1970)', 'The Tin Drum (1979)',
       'Murder on the Orient Express (1974)', 'The Favourite (2018)'],
      dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Raise the Red Lantern (1991)': <Candidate('Raise the Red Lantern (1991)')>,
 'Quills (2000)': <Candidate('Quills (2000)')>,
 'The Enigma of Kaspar Hauser (1974)': <Candidate('The Enigma of Kaspar Hauser (1974)')>,
 'Black Knight (2001)': <Candidate('Black Knight (2001)')>,
 'Waterloo (1970)': <Candidate('Waterloo (1970)')>,
 'The Tin Drum (1979)': <Candidate('The Tin Drum (1979)')>,
 'Murder on the Orient Express (1974)': <Candidate('Murder on the Orient Express (1974)')>,
 'The Favourite (2018)': <Candidate('The Favourite (2018)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Raise the Red Lantern (1991), Quills (2000), The Enigma of Kaspar Hauser (1974), Black Knight (2001), Waterloo (1970), The Tin Drum (1979), Murder on the Orient Express (1974), The Favourite (2018))>,
 <Ballot(Murder on the Orient Express (1974), The Favourite (2018), Raise the Red Lantern (1991), Quills (2000), Black Knight (2001), The Enigma of Kaspar Hauser (1974), The Tin Drum (1979), Waterloo (1970))>,
 <Ballot(The Favourite (2018), Murder on the Orient Express (1974), Raise the Red Lantern (1991), Waterloo (1970), Black Knight (2001))>,
 <Ballot(Waterloo (1970), Quills (2000), The Enigma of Kaspar Hauser (1974), Raise the Red Lantern (1991), The Tin Drum (1979), Murder on the Orient Express (1974), The Favourite (2018), Black Knight (2001))>,
 <Ballot(The Enigma of Kaspar Hauser (1974), Raise the Red Lantern (1991), The Tin Drum (1979), Waterloo (1970), Murder on the Orient Express (1974))>,
 <Ballot(Black Knight (2001), The Favourite (2018), Waterloo (1970))>,
 <Ballot(

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                              Votes  Status
-----------------------------------  -------  --------
The Favourite (2018)                       2  Hopeful
Raise the Red Lantern (1991)               2  Hopeful
Waterloo (1970)                            1  Hopeful
Murder on the Orient Express (1974)        1  Hopeful
The Enigma of Kaspar Hauser (1974)         1  Hopeful
The Tin Drum (1979)                        1  Hopeful
Black Knight (2001)                        1  Rejected
Quills (2000)                              0  Rejected

ROUND 2
Candidate                              Votes  Status
-----------------------------------  -------  --------
The Favourite (2018)                       3  Hopeful
Raise the Red Lantern (1991)               2  Hopeful
Waterloo (1970)                            1  Hopeful
The Enigma of Kaspar Hauser (1974)         1  Hopeful
The Tin Drum (1979)                        1  Hopeful
Murder on the Orient Express (1974)        1  Rejected
Black Kn

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
ROM COMS                           2  Hopeful
ANIMALS/CREATURES THAT TALK        2  Hopeful
MIAMI                              1  Hopeful
EURO-CRIMES                        1  Hopeful
SEX PERVERT MOVIES                 1  Hopeful
1980S ACTION                       1  Hopeful
[Default to Nicolas Cage]          1  Rejected
OOPS ALL STEVE BUSCEMI             0  Rejected
I'M WITH THE BAND                  0  Rejected

ROUND 2
Candidate                      Votes  Status
---------------------------  -------  --------
ROM COMS                           2  Hopeful
ANIMALS/CREATURES THAT TALK        2  Hopeful
1980S ACTION                       2  Hopeful
MIAMI                              1  Hopeful
EURO-CRIMES                        1  Hopeful
SEX PERVERT MOVIES                 1  Rejected
[Default to Nicolas Cage]          0  Rejected
OOPS ALL STEVE BUSCEMI             0  Rejected
I'M WITH TH